# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math


## Load the Data

Start by importing the data from the pickle file.

In [2]:
# TODO: Implement load the data here.
with open('data/train.p', 'rb') as f:
    train_data = pickle.load(f)

with open('data/test.p', 'rb') as f:
    test_data = pickle.load(f)

In [3]:
X_train = train_data['features']
y_train = train_data['labels']

X_test = test_data['features']
y_test = test_data['labels']

In [4]:
n_classes = len(set(y_train))

In [12]:
image_width = X_train[0].shape[0]
image_height = X_train[0].shape[1]
image_shape_flat = image_width * image_height

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [6]:
# TODO: Use `train_test_split` here.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.75, stratify=y_train)

In [7]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [8]:
def convert_image_to_gray(data):
    # Convert image to grayscale
    return np.mean(data, axis=3)
    
def preprocess_data(data):
    data_min = np.min(data)
    data_max = np.max(data)
    b = 0.5
    a = -0.5
    
    data = ((b-a) * (data - data_min)/(data_max - data_min)) + a
    # PCA and whitening?
    return data

# One hot encode labels
def encode_labels(labels):
    labels = (np.arange(n_classes) == labels[:,None]).astype(np.float)
    return labels

In [9]:
X_train = preprocess_data(convert_image_to_gray(X_train))
X_val = preprocess_data(convert_image_to_gray(X_val))
X_test = preprocess_data(convert_image_to_gray(X_test))

y_train_one_hot = encode_labels(y_train)
y_val_one_hot = encode_labels(y_val)
y_test_one_hot = encode_labels(y_test)

In [10]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

In [16]:
X_train = X_train.reshape(-1, image_shape_flat)
X_val = X_val.reshape(-1, image_shape_flat)
X_test = X_test.reshape(-1, image_shape_flat)

In [30]:
X_train.shape

(29406, 1024)

In [31]:
y_train.shape

(29406,)

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [25]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential

hidden_layer_1_size = 128


model = Sequential()

model.add(Dense(output_dim=hidden_layer_1_size, input_dim=image_shape_flat))
model.add(Activation("relu"))
model.add(Dense(output_dim=n_classes))
model.add(Activation("softmax"))



In [26]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 1024))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [27]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_5 (None, 1024) (None, 128) <function linear at 0x14cc4a510>
activation_5 (None, 128) (None, 128) <function relu at 0x14cc4a2f0>
dense_6 (None, 128) (None, 43) <function linear at 0x14cc4a510>
activation_6 (None, 43) (None, 43) <function softmax at 0x14cc4a048>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [41]:
# TODO: Compile and train the model here.
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
history = model.fit(X_train, y_train.reshape((-1, 1)), nb_epoch=20, validation_data=(X_val, y_val))


Train on 29406 samples, validate on 9803 samples
Epoch 1/20
29406/29406 [==============================] - 4s - loss: 0.3085 - acc: 0.9368 - val_loss: 0.4017 - val_acc: 0.9007
Epoch 2/20
29406/29406 [==============================] - 4s - loss: 0.3042 - acc: 0.9370 - val_loss: 0.3880 - val_acc: 0.9067
Epoch 3/20
29406/29406 [==============================] - 4s - loss: 0.3009 - acc: 0.9388 - val_loss: 0.3762 - val_acc: 0.9139
Epoch 4/20
29406/29406 [==============================] - 4s - loss: 0.2965 - acc: 0.9390 - val_loss: 0.3792 - val_acc: 0.9114
Epoch 5/20
29406/29406 [==============================] - 4s - loss: 0.2919 - acc: 0.9395 - val_loss: 0.3708 - val_acc: 0.9151
Epoch 6/20
29406/29406 [==============================] - 4s - loss: 0.2885 - acc: 0.9407 - val_loss: 0.3685 - val_acc: 0.9143
Epoch 7/20
29406/29406 [==============================] - 4s - loss: 0.2845 - acc: 0.9405 - val_loss: 0.3812 - val_acc: 0.9093
Epoch 8/20
29406/29406 [==============================] - 4s -

In [42]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 91%

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [52]:
X_train = X_train.reshape((-1, 32, 32, 1))
X_val = X_val.reshape((-1, 32, 32, 1))
X_test = X_test.reshape((-1, 32, 32,1 ))

y_train = y_train.reshape((-1, 1))
y_val = y_val.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

In [53]:
batch_size = 128
nb_classes = n_classes
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 32, 32
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)


In [54]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=(img_rows, img_cols, 1)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, y_train_one_hot, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_val, y_val_one_hot))

Train on 29406 samples, validate on 9803 samples
Epoch 1/12
29406/29406 [==============================] - 88s - loss: 2.4092 - acc: 0.3615 - val_loss: 0.8257 - val_acc: 0.7678
Epoch 2/12
29406/29406 [==============================] - 90s - loss: 0.7608 - acc: 0.7831 - val_loss: 0.3195 - val_acc: 0.9244
Epoch 3/12
29406/29406 [==============================] - 85s - loss: 0.4513 - acc: 0.8716 - val_loss: 0.2073 - val_acc: 0.9577
Epoch 4/12
29406/29406 [==============================] - 85s - loss: 0.3388 - acc: 0.9041 - val_loss: 0.1608 - val_acc: 0.9653
Epoch 5/12
29406/29406 [==============================] - 100s - loss: 0.2817 - acc: 0.9191 - val_loss: 0.1352 - val_acc: 0.9694
Epoch 6/12
29406/29406 [==============================] - 96s - loss: 0.2427 - acc: 0.9308 - val_loss: 0.1112 - val_acc: 0.9732
Epoch 7/12
29406/29406 [==============================] - 89s - loss: 0.2064 - acc: 0.9393 - val_loss: 0.0980 - val_acc: 0.9765
Epoch 8/12
29406/29406 [==============================

In [55]:
score = model.evaluate(X_test, y_test_one_hot, verbose=0)
print("Test Accuracy: ", score)

Test Accuracy:  [0.2597106642106744, 0.94243863815366513]


In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [ ]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

# TODO: Compile and train the model here.

In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [ ]:
# TODO: Re-construct the network and add dropout after the pooling layer.

# TODO: Compile and train the model here.

In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [ ]:
# TODO: Load test data
    
# TODO: Preprocess data & one-hot encode the labels

# TODO: Evaluate model on test data

**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.